### Conditional Chain

In [104]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, ValidationError
from typing import Literal
load_dotenv()

True

In [105]:
model = ChatGroq(model='llama-3.1-8b-instant')

In [106]:
parser = StrOutputParser()

In [107]:
class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')


In [108]:
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [109]:
prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text. Output must be a JSON in the following format:\n{format_instruction}\nFeedback: {feedback}',
    input_variables=["feedback"],
    partial_variables={"format_instruction": parser2.get_format_instructions()}
)

In [110]:
classifier_chain = prompt1 | model | parser2

In [111]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

In [112]:
prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

In [113]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [114]:
chain = classifier_chain | branch_chain

In [115]:
print(chain.invoke({'feedback': 'This is a beautiful phone'}))

"Thank you so much for your kind words! I'm glad I could help and appreciate your feedback. It means a lot to me!"


In [116]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
